# <center><font color=#76B900 size="+3"><b>**NVIDIA cuOpt for CVRPTW**</b></font></center>
---

**Learning Objectives:**
- Introducing multiple cosntrints to model more complex problems
- Using real-world data and running data preprocessing
- Integrating third-party tools for traffic data
  
In this notebook, we will use cuOpt to solve a real world application of a Capacitated Vehicle Routing Problem with Time Windows (CVRPTW). This use case is also called Last Mile Delivery (LMD).

Suppose you are working as an Optimization Scientist at a grocery chain throughout New York City. There are 97 stores and 3 distribution centers. Every day, stores place an order for food that must be delivered the next day to ensure they are fully stocked. Given input data about stores' orders, distribution centers, and the available fleet of vehicles, it is your job to calculate the route for each vehicle such that all orders are fulfilled while minimizing vehicles' travel time and cost. For a problem space of 100 locations there are about 100! possible routes. You can do the math yourself- but that's a lot. Lucky for you, you have access to the cuOpt solver. All you need to do is read the input data and preprocess the data. Once all the data is ready, you just need to save it to one Python dictionary and send it to cuOpt, which does all the hard computation for you.

Let's walk through these steps. 

For the Last Mile Delivery (LMD) use case, we need 3 datasets with the following features:

- Depots
    - Name
    - Location
    - Start and end time (operation hours)
- Orders
    - Store Name
    - Location
    - Start and end time (store hours)
    - Demand
    - Service time
    - Loyalty Member
    - Delivery Requirement
- Vehicles
    - Name/ID Number
    - Assigned depot
    - Start and end time (vehicle/driver shift hours)
    - Break time
    - Capacity
    - Max time
    
You may have additional features depending on the problem at hand.


In [1]:
#import dependencies
import pandas as pd
from IPython.display import HTML
import requests
import json
from scipy.spatial import distance

from helper_function.notebook_helpers import show_vehicle_routes, get_minutes_from_datetime
from helper_function.map_helpers import get_map_by_vehicle

# from ../tmp/cuopt_managed_service/cuopt_thin_client import CuOptServiceClient

<br>

## Read input data from CSV files

Here, we are using real world addresses instead of points in the Euclidean space. Location data needs to be in the form of coordinate points (longitude and latitude values). Our data already has coordinate points included. If you're using new data and need to do so yourself, you will need to use a third party tool.

In this workflow, we are using locations from the following [Kaggle dataset](https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata). This is a dataset of Airbnbs in New York City. Our problem space has 100 locations total which includes 3 depots and 97 orders. The coordinate points are taken from the dataset and the rest of the features are synthetic data. We have 15 vehicles available.

In [2]:
DATA_PATH = "data/"

orders_df = pd.read_csv(DATA_PATH+"orders_lmd.csv")
depots_df = pd.read_csv(DATA_PATH+"depots_lmd.csv")
vehicles_df = pd.read_csv(DATA_PATH+"vehicles_lmd.csv")

In [3]:
n_depots = len(depots_df.index)
n_orders = len(orders_df.index)
n_vehicles = len(vehicles_df.index)

n_loc_total = n_orders + n_depots

In [4]:
locations_df = (pd.concat([depots_df[["Name","Longitude","Latitude"]], orders_df[["Name","Longitude","Latitude"]]], ignore_index=True)).reset_index()

<br>

## Create Cost Matrix

The **cost matrix** models the cost between each pair of locations.  It is used by cuOpt to compute the cost of traveling from any location to any other. The cost matrix needs to be a square matrix of dimension equal to the total number of locations which inlcludes both depots and orders. In this Vehicle Routing Problem, our cost metric is travel time. This is cost we want to minimize. 

To build a a cost matrix of live traffic data, we need to use a third party map data provider. In this workflow, the cost matrix will calculate the travel time in minutes between each two pairs of locations which we build using [OSRM](https://project-osrm.org/).


In practical applications, you can integrate this to a third-party map data provider like Esri or Google Maps to get live traffic data and run dynamic/real-time re-routing using cuOpt.

In [13]:
 def build_travel_time_matrix(df):
    latitude = df.Latitude.to_numpy()
    longitude = df.Longitude.to_numpy()
    
    locations=""
    n_orders = len(df)
    for i in range(n_orders):
        locations = locations + "{},{};".format(longitude[i], latitude[i])
    r = requests.get("http://router.project-osrm.org/table/v1/car/"+ locations[:-1])
    routes = json.loads(r.content)
    
    # OSRM returns duration in seconds. Here we are converting to minutes
    for i in routes['durations']:
        i[:] = [x / 60 for x in i]
    
    coords_index = { i: (latitude[i], longitude[i]) for i in range(df.shape[0])}
    time_matrix = pd.DataFrame(routes['durations'])
    
    return time_matrix

In [14]:
cost_matrix_df = build_travel_time_matrix(locations_df)

<br>

## Preprocess Fleet and Task Data

Here, we take our raw input data and convert it to the format needed for cuOpt.

### Set Fleet Data

Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- vehicle_locations is a list of the start and end location of the vehicles. Each vehicle is assigned to a depot from which it departs in the morning and returns to at night. For example, a vehicle that starts and ends in depot 1 which is the location at index 0 would have the vehicle location of [0,0]. 

- capacities is a list of how much goods each vehicle can carry in weight. Here we have two different types of vehicles: trucks and EV vans. A truck can carry up to 20,000 pounds and an EV van can carry up to 8,000 pounds. This is essential when assigning orders to vehicles because one vehicle can only carry so many orders at once. 

- vehicle_time_windows is a list of the integer representation of the operating time of each vehicle. Equivalently, the shift of each vehicle driver. We convert the UTC timestamp to epoch time (integer representation in minutes).

- vehicle_break_time_windows is a list of the integer representation of break time of each vehicle within its operating time. For a driver working an 8 hour shift, this break in the middle of the day represents their lunch break. These time windows are when their lunch break may occur.
  
- vehicle_break_durations is the length of the break. Here, we set the duration to be 30 minutes for all vehicles. 

- vehicle_max_time is a list of the maximum time a vehicle can operate. Even if a driver is available for a long period of time, this constraint enforces a maximum length for a driver's shift. This is also given in minutes. A driver's time window represents total availability which may be longer than a standard shift length. If a driver says they are available to work from 9am to 9pm, we still want to limit their shift to be shorter. A truck driver can drive up to 7 hours, and an EV driver can drive up to 4 hours. 

In [6]:
depot_names_to_indices_dict = {locations_df["Name"].values.tolist()[i]: i for i in range(n_depots)}
vehicle_locations = vehicles_df[["assigned_depot","assigned_depot"]].replace(depot_names_to_indices_dict).values.tolist()

In [7]:
capacities = [[int(a) for a in vehicles_df['vehicle_capacity'].tolist()]]

In [8]:
vehicle_time_windows = pd.concat((vehicles_df['vehicle_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['vehicle_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [9]:
vehicle_break_time_windows = [pd.concat((vehicles_df['break_start'].apply(get_minutes_from_datetime).to_frame(), vehicles_df['break_end'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()]

In [10]:
vehicle_break_durations = [[30] * n_vehicles]

In [11]:
vehicles_max_time = vehicles_df['max_time'].tolist()

### Set Task Data


Here we take our raw data from the csv file and convert it into data that we can send to the cuOpt solver.

- task_locations is the list of stores that have placed an order. This list is simply the index of each location. 

- task_time_windows is the list of integer representation of opening hours for each store. We convert the UTC timestamp to epoch time (integer representation in minutes).

- service_times is the list of the length of time for orders to be dropped off once the vehicle reaches the location. Here, these values are between 15 and 30 minutes.

- demand is the list of weight demand for each order. Here, these values are between 40 and 200 pounds. 

- vehicle_match_list allows us to ensure that some orders are assigned to specific vehicles. In this use case, some of the orders are frozen and can be delivered in trucks and not EV vans. Here we can indicate that the frozen orders are assigned specifically to vehicles that are trucks.  

In [15]:
task_locations = locations_df.index.tolist()[n_depots:]

In [16]:
demands = [orders_df['Demand'].values.tolist()]

In [17]:
service_times = orders_df['ServiceTime'].values.tolist()

In [18]:
task_time_windows = pd.concat((orders_df['order_start_time'].apply(get_minutes_from_datetime).to_frame(), orders_df['order_end_time'].apply(get_minutes_from_datetime).to_frame()), axis=1).values.tolist()

In [19]:
trucks_ids = vehicles_df['vehicle_type'][vehicles_df['vehicle_type']=="Truck"].index.values.tolist()

In [20]:
vehicle_match_list = []
for i in orders_df['is_frozen'][orders_df['is_frozen']==1].index.values.tolist():
    vehicle_match_list.append({"order_id": i, "vehicle_ids": trucks_ids})

<br>

## Save data in a dictionary

Here, we take all the data we have prepared so far and save it to one dictionary. This includes the cost matrices, task data, fleet data, and solver config. This is all the data that cuOpt needs to solve our LMD problem. 

In [34]:
cuopt_problem_data = {
    "cost_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.to_numpy().tolist()
        }
    },
    "travel_time_matrix_data": {
        "cost_matrix": {
            "0": cost_matrix_df.to_numpy().tolist()
        }
    },
    "task_data": {
        "task_locations": task_locations,
        "demand": demands,
        "task_time_windows": task_time_windows,
        "service_times":service_times,
        "order_vehicle_match": vehicle_match_list,
    },
    "fleet_data": {
        "vehicle_locations": vehicle_locations,
        "capacities": capacities,
        "vehicle_time_windows": vehicle_time_windows,
        "vehicle_break_time_windows": vehicle_break_time_windows,
        "vehicle_break_durations": vehicle_break_durations,
        "vehicle_max_times": vehicles_max_time,
    },
    "solver_config": {
        "time_limit": 0.1,
    }
}

In [35]:
import json
# Convert and write JSON object to file
with open("cuopt_data_lmd.json", "w") as outfile: 
    json.dump(cuopt_problem_data, outfile)


<br>

## Create a Service Client Instance

Now that we have prepared all of our data, we can establish a connection to the cuOpt service. 

In the cell below, paste your client ID and Secret that you received via email in order to authenticate the connection. 

Here, we create an instance of the cuOpt Service Client to establish a connection. 


In [36]:
# need to change once we connect to educational client
 
cuopt_client_id = "nvssa-prd-Pwqn82nr9tjKCn_L0XLaiGUjFC-jbsaUYrm0T7Y7J14"
cuopt_client_secret = "ssap-B6kzpkUQQ3YcfCU"

cuopt_service_client = CuOptServiceClient(
    client_id=cuopt_client_id,
    client_secret=cuopt_client_secret,
    )

NameError: name 'CuOptServiceClient' is not defined

<br>

## Make the cuOpt API call
---

In [37]:
# need to change once we connect to educational client

!python /Users/adigeva/Desktop/cuOpt/route-opt-workflow/route-opt-workflow-env/lib/python3.10/site-packages/cuopt_thin_client/cuopt_cli.py cuopt_data_lmd.json -c ../cuopt_thin_client/credentials.json

2024-01-05 14:33:37.441 cuopt_thin_client.cuopt_service_client INFO Using Cached Token
{'response': {'solver_response': {'status': 0, 'num_vehicles': 10, 'solution_cost': 376.989990234375, 'vehicle_data': {'0': {'task_id': ['Depot', '57', '8', '66', '30', '91', 'Break', 'Depot'], 'arrival_stamp': [661.806640625, 664.2100219726562, 687.856689453125, 707.7433471679688, 731.7766723632812, 761.4033203125, 781.0, 811.0], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Depot'], 'route': [0, 60, 11, 69, 33, 94, 0, 0]}, '1': {'task_id': ['Depot', '85', '19', '38', '54', '77', '45', 'Break', '31', '72', '67', '95', '48', '70', '65', '24', 'Depot'], 'arrival_stamp': [570.0, 579.4966430664062, 612.6799926757812, 634.5083618164062, 663.538330078125, 688.5750122070312, 719.760009765625, 745.9116821289062, 775.9116821289062, 807.64501953125, 826.0416870117188, 851.32666015625, 877.4716796875, 903.9833374023438, 921.3316650390625, 943.6666870117188, 988.9333496

In [42]:
# Solve the problem
# solver_response = cuopt_service_client.get_optimized_routes(
#     cuopt_problem_data
# )

# Process returned data
solver_resp = {'response': {'solver_response': {'status': 0, 'num_vehicles': 10, 'solution_cost': 376.989990234375, 'vehicle_data': {'0': {'task_id': ['Depot', '57', '8', '66', '30', '91', 'Break', 'Depot'], 'arrival_stamp': [661.806640625, 664.2100219726562, 687.856689453125, 707.7433471679688, 731.7766723632812, 761.4033203125, 781.0, 811.0], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Depot'], 'route': [0, 60, 11, 69, 33, 94, 0, 0]}, '1': {'task_id': ['Depot', '85', '19', '38', '54', '77', '45', 'Break', '31', '72', '67', '95', '48', '70', '65', '24', 'Depot'], 'arrival_stamp': [570.0, 579.4966430664062, 612.6799926757812, 634.5083618164062, 663.538330078125, 688.5750122070312, 719.760009765625, 745.9116821289062, 775.9116821289062, 807.64501953125, 826.0416870117188, 851.32666015625, 877.4716796875, 903.9833374023438, 921.3316650390625, 943.6666870117188, 988.933349609375], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Depot'], 'route': [2, 88, 22, 41, 57, 80, 48, 34, 34, 75, 70, 98, 51, 73, 68, 27, 2]}, '2': {'task_id': ['Depot', '7', '75', '44', '15', 'Break', '32', '41', '56', 'Depot'], 'arrival_stamp': [690.5416870117188, 695.9533081054688, 729.9083251953125, 764.0833129882812, 790.0633544921875, 818.0, 848.0, 869.8250122070312, 897.6683349609375, 929.5133056640625], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Delivery', 'Delivery', 'Depot'], 'route': [0, 10, 78, 47, 18, 35, 35, 44, 59, 0]}, '3': {'task_id': ['Depot', 'Break', '26', '12', '25', '11', '86', '27', '33', 'Depot'], 'arrival_stamp': [738.0, 738.0, 769.3833618164062, 792.0616455078125, 818.8099975585938, 841.1166381835938, 872.143310546875, 891.2566528320312, 922.5133056640625, 951.9849853515625], 'type': ['Depot', 'Break', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Depot'], 'route': [1, 1, 29, 15, 28, 14, 89, 30, 36, 1]}, '4': {'task_id': ['Depot', '5', '83', '84', '18', 'Break', '79', '76', '47', '10', '88', '94', '71', '55', '87', '60', '63', 'Depot'], 'arrival_stamp': [618.6849975585938, 622.4266967773438, 653.5616455078125, 674.219970703125, 692.0, 711.0, 741.7133178710938, 766.4933471679688, 783.4666748046875, 801.6616821289062, 829.719970703125, 858.1283569335938, 881.094970703125, 910.60498046875, 928.9183349609375, 959.4166870117188, 991.5066528320312, 1024.3717041015625], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Depot'], 'route': [2, 8, 86, 87, 21, 21, 82, 79, 50, 13, 91, 97, 74, 58, 90, 63, 66, 2]}, '5': {'task_id': ['Depot', '50', '58', '81', '51', '52', '64', '0', 'Break', '82', 'Depot'], 'arrival_stamp': [628.5816650390625, 630.0, 662.8599853515625, 681.8200073242188, 711.6466674804688, 730.719970703125, 756.8416748046875, 771.8599853515625, 794.3633422851562, 824.3633422851562, 850.7366943359375], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Depot'], 'route': [2, 53, 61, 84, 54, 55, 67, 3, 85, 85, 2]}, '6': {'task_id': ['Depot', '68', '39', '42', '73', '93', '40', '53', '49', 'Break', '90', '69', '3', '4', '46', '2', 'Depot'], 'arrival_stamp': [555.0, 561.135009765625, 601.52001953125, 631.9600219726562, 651.3433227539062, 685.0816650390625, 705.3333129882812, 735.0549926757812, 756.4266967773438, 780.4266967773438, 813.5616455078125, 835.2433471679688, 863.780029296875, 891.3866577148438, 907.211669921875, 935.3966674804688, 967.5733032226562], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Depot'], 'route': [2, 71, 42, 45, 76, 96, 43, 56, 52, 52, 93, 72, 6, 7, 49, 5, 2]}, '7': {'task_id': ['Depot', '23', '16', '92', '14', '1', '96', 'Break', '9', 'Depot'], 'arrival_stamp': [653.5966796875, 655.8099975585938, 690.5333251953125, 714.3316650390625, 737.3533325195312, 759.8333129882812, 793.0, 818.0, 861.4400024414062, 888.288330078125], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Depot'], 'route': [0, 26, 19, 95, 17, 4, 99, 99, 12, 0]}, '8': {'task_id': ['Depot', '28', '34', '74', '36', '62', 'Break', 'Depot'], 'arrival_stamp': [597.193359375, 600.0, 620.239990234375, 646.7150268554688, 674.6400146484375, 703.64501953125, 727.0650024414062, 762.3699951171875], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Depot'], 'route': [2, 31, 37, 77, 39, 65, 74, 2]}, '9': {'task_id': ['Depot', '13', '43', '22', '17', '59', '61', '35', '29', 'Break', '6', '89', '80', '78', '20', '37', '21', 'Depot'], 'arrival_stamp': [555.0, 557.181640625, 592.4266967773438, 614.2816772460938, 641.2633056640625, 672.5433349609375, 691.8300170898438, 721.8049926757812, 743.1300048828125, 766.1300048828125, 796.2050170898438, 821.3516845703125, 841.4500122070312, 864.5533447265625, 885.9216918945312, 912.5466918945312, 939.92333984375, 974.6233520507812], 'type': ['Depot', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Break', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Delivery', 'Depot'], 'route': [0, 16, 46, 25, 20, 62, 64, 38, 32, 32, 9, 92, 83, 81, 23, 40, 24, 0]}}}}, 'reqId': '45078585-416b-450c-af5e-d4fd7e76d65f'}
solver_resp = solver_resp["response"]["solver_response"]

location_names = [str(x) for x in locations_df.index.tolist()]

if solver_resp["status"] == 0:
    print("Cost for the routing in distance: ", solver_resp["solution_cost"])
    print("Vehicle count to complete routing: ", solver_resp["num_vehicles"])
    show_vehicle_routes(solver_resp, location_names)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", solver_resp["status"])

Cost for the routing in distance:  376.989990234375
Vehicle count to complete routing:  10
For vehicle - 0 route is: 

0->60->11->69->33->94->0->0


For vehicle - 1 route is: 

2->88->22->41->57->80->48->34->34->75->70->98->51->73->68->27->2


For vehicle - 2 route is: 

0->10->78->47->18->35->35->44->59->0


For vehicle - 3 route is: 

1->1->29->15->28->14->89->30->36->1


For vehicle - 4 route is: 

2->8->86->87->21->21->82->79->50->13->91->97->74->58->90->63->66->2


For vehicle - 5 route is: 

2->53->61->84->54->55->67->3->85->85->2


For vehicle - 6 route is: 

2->71->42->45->76->96->43->56->52->52->93->72->6->7->49->5->2


For vehicle - 7 route is: 

0->26->19->95->17->4->99->99->12->0


For vehicle - 8 route is: 

2->31->37->77->39->65->74->2


For vehicle - 9 route is: 

0->16->46->25->20->62->64->38->32->32->9->92->83->81->23->40->24->0




<br>

## Visualize the Routes

In the drop down menu below, you can select different vehicle ID's to see if they are dispatched. If they are, we print their assigned route on a map.


Generating a route and map uses third party tools and takes about 30 seconds to run.

In [43]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets
from ipywidgets import interact

w = widgets.Dropdown(
    options = list(vehicles_df.index.values),
    description='Vehicle ID:',
)

def on_change(value):
    if str(value) in list(solver_resp['vehicle_data'].keys()):
        curr_route_df = pd.DataFrame(solver_resp["vehicle_data"][str(value)]['route'], columns=["stop_index"])
        curr_route_df = pd.merge(curr_route_df, locations_df, how="left", left_on=["stop_index"], right_on=["index"])
        display(get_map_by_vehicle(curr_route_df))        
    else:
        print("This Vehicle is not assigned to any order!!")

interact(on_change, value=w)

interactive(children=(Dropdown(description='Vehicle ID:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0), Ou…

<function __main__.on_change(value)>